In [1]:
import numpy as np
import pandas as pd
import cv2
import glob
from multiprocessing import Pool
import matplotlib.pyplot as plt
import time
import os
import itertools
from scipy.stats import multivariate_normal

In [2]:
#Dataset name
#Path of the data folder
#Path where we want to save the file 
#Number of previous frames
#probability threshold for declaring as background
#probability for false detaction suppression
#connected components area for intersection with the probability
#minimum area of connected components to be considered
DataSet="HallAndMonitor"
PathFolder="Datasets/"+DataSet+"/"+DataSet+"/input/"
Result="Results/"+DataSet+"/"
N=40
th1=1e-5
prob_threshold=th1
components_area=200
component_size_threshold_min=50

In [3]:
def PerPixelCalculationKernelDensity(data):
    
    data=data.reshape(-1)
    n_frames=N
    prevFrames=data[C:C*(n_frames)+C]
    diffMatrix=[]
    Prob=0

    #calculating the difference in pixel values between the frames
    for f in range(0,n_frames-1):
        diffMatrix.append(np.abs(prevFrames[f*C:f*C+C]-prevFrames[(f+1)*C:(f+1)*C+C]))

    variance=np.median(np.array(diffMatrix),axis=0)
    variance=variance/(0.68*np.sqrt(2))
    variance[variance==0]=1.0
    #variance=np.ones(C)
    
    for f in range(0,n_frames):
        Prob+=multivariate_normal.pdf(data[:C],prevFrames[f*C:f*C+C],np.diag(variance))

    Prob=Prob/n_frames
    output_mask=255.0

    if Prob>th1:
        data[:C]=255*np.ones(C)
        output_mask=0.0   

    data[-(C+1)]=output_mask
    data[-C:]=variance
    return data

In [4]:
def draw_bounding_boxes(image,boxes):
    for box in boxes:
        image = cv2.rectangle(image,box[:2], box[2:], (255,0,0), 1)
    return image

In [5]:
def FalseDetectionSupression(mask,img,variance,prob_threshold=1e-5,component_size_threshold_min=component_size_threshold_min,component_size_threshold=components_area, neighbourhood_size=(2,2)):
    mask[mask==255]=1
    mask=np.uint8(mask)
    ConnectedComponents=cv2.connectedComponentsWithStats(mask,4,cv2.CV_32S)
    component_labels=ConnectedComponents[1]
    connected_components_area=ConnectedComponents[2][:,-1]
    
    for x in range(mask.shape[0]):
        for y in range(mask.shape[1]):    
            if mask[x][y]==0:
                continue
            if connected_components_area[component_labels[x][y]] < component_size_threshold_min:
                mask[x][y]=0
                continue
            maxProb=-1
            isReach=False
            for neigh_x in range(max(0,x-neighbourhood_size[0]),min(mask.shape[0]-1,x+neighbourhood_size[0])):
                for neigh_y in range(max(0,y-neighbourhood_size[1]),min(mask.shape[1]-1,y+neighbourhood_size[1])):
                    if neigh_x==x and neigh_y==y:
                        continue
                    maxProb=max(multivariate_normal.pdf(img[x,y,:],img[neigh_x,neigh_y,:],variance[neigh_x,neigh_y,:]),maxProb)
                    if maxProb>prob_threshold:
                        isReach=True
                        break
                if isReach:
                    break
            if maxProb>prob_threshold and connected_components_area[component_labels[x][y]] < component_size_threshold:
                mask[x][y]=0
    
    boxes=[]
    for bb in ConnectedComponents[2]:
        if bb[-1]>component_size_threshold:
            boxes.append([bb[0],bb[1],bb[0]+bb[2],bb[1]+bb[3]])
    mask[mask==1]=255
    img=SegmentationImage(img,mask.copy())
    mask=draw_bounding_boxes(mask,boxes)
    img=draw_bounding_boxes(img,boxes)
    return mask,img

In [6]:
def SegmentationSharpeningMask(mask,window_size=(32,32),step_size=(16,16), threshold=100):
    window_width = window_size[0]
    window_height = window_size[1]
    
    for x in range(0, mask.shape[0], step_size[0]):
        for y in range(0, mask.shape[1], step_size[1]):
            window = mask[x:x+window_width, y:y+window_height]
            non_zero_pixels = cv2.countNonZero(window)
            if non_zero_pixels < threshold:
                mask[x:x+window_width, y:y+window_height] = 0.0
    return mask

In [7]:
def SegmentationImage(img,mask):
    mask[mask==255]=1
    mask_3d=np.repeat(mask[:, :, np.newaxis], 3, axis=2)
    masked_img=back_less_img*mask_3d
    masked_img[masked_img==0]=255
    return masked_img    

In [8]:
#(PerPixelCalculation(np.array([122,122,122,1.0, 1.0,0.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0,0.0, 400.0,0.0,0.0,0.0,0.0, 125.0,125.0,125.0, 0.0,0.0,0.0 ,0.0,0.0,0.0 ,0.0,0.0,0.0 ,0.0,0.0,0.0, 1.0])))

In [9]:
#N=4
#PerPixelCalculationKernelDensity(np.array([122.0,122.0,122.0, 122.0,122.0,122.0 , 123.0,124.0,124.0 ,124.0,124.0,124.0,124.0,124.0,124.0, 0, 0.0,0.0,0.0]))

In [10]:
images = []
img_dir=[]
for img in glob.glob(PathFolder+"*.png"):
    img_dir.append(img)
img_dir=np.sort([i for i in img_dir if i.endswith(".png")])


In [11]:
for img in img_dir:
    images.append(cv2.imread(img))
width=images[0].shape[0]
height=images[0].shape[1]


In [12]:
# current frame + N frames before the current frame + output_mask + Variances in each pixel 
C=images[0].shape[2]
z_axis= C + N*C + 1 + C 
dataMatrix=np.zeros((width,height,z_axis))

In [14]:
p = Pool(processes=15)
fourcc1 = cv2.VideoWriter_fourcc(*'mp4v')
out_video = cv2.VideoWriter(Result+'output_high_thres'+DataSet+'.avi', fourcc1, 10, (height, width))

In [ ]:
start_time=time.time()
mask_list=[]
img_list=[]
for i in range(len(images)):
    dataMatrix[:,:,:C]=images[i]
    inputData = np.split(dataMatrix.reshape(width*height,z_axis), width*height)
    returnData = p.map(PerPixelCalculationKernelDensity, inputData)
    returnData=np.array(returnData).reshape((width,height,z_axis))

    n_frames=N
    output_mask=returnData[:,:, -(C+1)]
    back_less_img= returnData[:,:,:C]
    variance=returnData[:,:,-C:]

    suppressed_mask,suppressed_img=FalseDetectionSupression(output_mask.copy(),back_less_img.copy(),variance,prob_threshold=prob_threshold)
    
    dataMatrix[:,:, C : (n_frames*C)]=returnData[:,:,2*C: (n_frames*C)+C]
    dataMatrix[:,:,(n_frames*C):(n_frames*C)+C]=images[i]
    out_video.write(np.uint8(suppressed_img))
    mask_list.append(suppressed_mask)
    print("Processed Frame "+str(i))

print("Finished processing in ", (time.time()-start_time)/60)    
cv2.destroyAllWindows()
out_video.release()

fourcc1 = cv2.VideoWriter_fourcc(*'mp4v')
out_mask = cv2.VideoWriter(Result+'output_mask_high'+DataSet+'.avi', fourcc1, 10, (height, width))
for i in mask_list:
    mask_3d=np.repeat(i[:, :, np.newaxis], 3, axis=2)
    out_mask.write(np.uint8(mask_3d))
out_mask.release()

Processed Frame 0
Processed Frame 1
Processed Frame 2
Processed Frame 3
Processed Frame 4
Processed Frame 5
Processed Frame 6
Processed Frame 7
Processed Frame 8
Processed Frame 9
Processed Frame 10
Processed Frame 11
Processed Frame 12
Processed Frame 13
Processed Frame 14
Processed Frame 15
Processed Frame 16
Processed Frame 17
Processed Frame 18
Processed Frame 19
Processed Frame 20
Processed Frame 21
Processed Frame 22
Processed Frame 23
Processed Frame 24
Processed Frame 25
Processed Frame 26
Processed Frame 27
Processed Frame 28
Processed Frame 29
Processed Frame 30
Processed Frame 31
Processed Frame 32
Processed Frame 33
Processed Frame 34
Processed Frame 35
Processed Frame 36
Processed Frame 37
Processed Frame 38
Processed Frame 39
Processed Frame 40
Processed Frame 41
Processed Frame 42
Processed Frame 43
Processed Frame 44
Processed Frame 45
Processed Frame 46
Processed Frame 47
Processed Frame 48
Processed Frame 49
Processed Frame 50
Processed Frame 51
Processed Frame 52
Pro

In [ ]:
#plt.imshow(np.uint8(output_mask.copy()),cmap="gray")

In [137]:
"""class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def intersection(a,b):    
    l1=Point(a[0],a[1])
    r1=Point(a[2],a[3])
    l2=Point(b[0],b[1])
    r2=Point(b[2],b[3])
    if l1.x > r2.x or l2.x > r1.x:
        return False
 
    if r1.y > l2.y or r2.y > l1.y:
        return False
 
    return True
  
def union(a,b):
  x = min(a[0], b[0])
  y = min(a[1], b[1])
    
  w = max(a[2], b[2]) 
  h = max(a[3], b[3]) 
  return (x, y, w, h)


def combine_boxes(boxes):
    new_array = []
    for boxa, boxb in itertools.combinations(boxes, 2):
        if intersection(boxa, boxb):
            new_array.append(union(boxa, boxb))
        else:
            new_array.append(boxa)
    return np.array(new_array).astype('int')"""


In [100]:
"""def runboxMerging(boxes,epochs=10):
    for i in range(epochs):
        boxes=boxMerging(boxes)
    return boxes

def boxMerging(boxes, overlapThresh = 0.05):
    if len(boxes) == 0:
        return []
    big_boxes=[]
    dp=np.zeros(len(boxes))
    for i in range(len(boxes)):
        if dp[i]==0:
            dp[i]=1
            minx=boxes[i][0]
            miny=boxes[i][1]
            maxx=boxes[i][2]
            maxy=boxes[i][3]
            for j in range(i+1,len(boxes)):
                if  dp[j]==0 and get_iou(boxes[i],boxes[j])> overlapThresh:
                    dp[j]=1
                    minx=min(minx,boxes[j][0])
                    miny=min(miny,boxes[j][1])
                    maxx=max(maxx,boxes[j][2])
                    maxy=max(maxy,boxes[j][3])
            big_boxes.append([minx,miny,maxx,maxy])
    return big_boxes
def get_iou(bb1, bb2):
    
    bb1_x1=bb1[1]
    bb1_y1=bb1[0]
    bb1_x2=bb1[3]
    bb1_y2=bb1[2]
    
    bb2_x1=bb2[1]
    bb2_y1=bb2[0]
    bb2_x2=bb2[3]
    bb2_y2=bb2[2]
    
    assert bb1_x1 < bb1_x2
    assert bb1_y1 < bb1_y2
    assert bb2_x1 < bb2_x2
    assert bb2_y1 < bb2_y2


    
    x_left = max(bb1_x1, bb2_x1)
    y_top = max(bb1_y1, bb2_y1)
    x_right = min(bb1_x2, bb2_x2)
    y_bottom = min(bb1_y2, bb2_y2)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1_x2 - bb1_x1) * (bb1_y2 - bb1_y1)
    bb2_area = (bb2_x2 - bb2_x1) * (bb2_y2 - bb2_y1)

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou
"""

In [ ]:
"""def FalseDetectionSuppression(img,mask,variance,window_size=(2,2),threshold=1e-3):
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if mask[x,y] == 0:
                continue
            pixel=img[x,y,:]
            isFound=False
            for i in range(max(0,x-window_size[0]),min(img.shape[0]-1,x+window_size[0])):
                for j in range(max(0,y-window_size[1]),min(img.shape[1]-1,y+window_size[1])):
                    if i==x and j==y:
                        continue
                    if multivariate_normal.pdf(pixel,img[i,j,:],np.eye(C)) > threshold:
                        mask[i,j]=0
                        isFound=True
                        break
                if isFound:
                    break
    return mask,SegmentationImage(img,mask)"""